In [1]:
import sys
sys.path.append('../../Modules/')
sys.path.append('../../Modules/Processors from Prof')
from Packages import *
from My_Json_processor import *

# Import processors from Prof
from ipynb.fs.full.Utilities import *
from ipynb.fs.full.Json_Processor import *
from ipynb.fs.full.CSV_Processor import *

In [2]:
main_df = ARC_Challenge_Processor_my('ARC-Challenge','ARC-Challenge-Test.jsonl')
main_df = main_df.convert_df()

In [3]:
main_df.head(20)

,target,input,choices
0,C,An astronomer observes that a planet rotates faster after a meteorite impact. Which is the most likely effect of this increase in rotation?,A: Planetary density will decrease.\nB: Planetary years will become longer.\nC: Planetary days will become shorter.\nD: Planetary gravity will become stronger.\n
1,B,A group of engineers wanted to know how different building designs would respond during an earthquake. They made several models of buildings and tested each for its ability to withstand earthquake conditions. Which will most likely result from testing different building designs?,A: buildings will be built faster\nB: buildings will be made safer\nC: building designs will look nicer\nD: building materials will be cheaper\n
2,C,The end result in the process of photosynthesis is the production of sugar and oxygen. Which step signals the beginning of photosynthesis?,A: Chemical energy is absorbed through the roots.\nB: Light energy is converted to chemical energy.\nC: Chlorophyll in the leaf captures light energy.\nD: Sunlight is converted into chlorophyll.\n
3,D,"A physicist wants to determine the speed a car must reach to jump over a ramp. The physicist conducts three trials. In trials two and three, the speed of the car is increased by 20 miles per hour. What is the physicist investigating when he changes the speed?",A: the control\nB: the hypothesis statement\nC: the dependent (responding) variable\nD: the independent (manipulated) variable\n
4,D,An astronaut drops a 1.0 kg object and a 5.0 kg object on the Moon. Both objects fall a total distance of 2.0 m vertically. Which of the following best describes the objects after they have fallen a distance of 1.0 m?,A: They have each lost kinetic energy.\nB: They have each gained the same amount of potential energy.\nC: They have each lost the same amount of potential energy.\nD: They have each gained one-half of their maximum kinetic energy.\n
5,B,Devil facial tumor disease (DFTD) is a disease that is decimating the population of Tasmanian devils. The disease passes from one animal to another through bites and is caused by parasites. The parasites cause cancerous tumors that spread throughout an infected animal's body and kill it. What is the best description of DFTD?,"A: a non-infectious, cell-cycle disease\nB: an infectious, cell-cycle disease\nC: a non-infectious, chronic disease\nD: an infectious, chronic disease\n"
6,C,"A type of small mammal from the mountain regions of the western United States makes its home out of piles of rock. During summer months, the mammal places grasses and seeds in protected places in the rock piles. Which of the following is the most likely reason for this behavior?",A: to repare for migration before winter\nB: to provide warmth during the cold winter months\nC: to store food that will be eaten over the winter months\nD: to protect the grasses and seeds from decay before winter\n
7,C,Petrified palm trees are found in sedimentary rock near glaciers. The presence of the petrified palm trees most likely provides evidence for which statement?,A: There was once more water in the area.\nB: The area was once grassland.\nC: The climate in the area was once tropical.\nD: There are active faults in the area.\n
8,B,Farmers in Wyoming were concerned because some of their chickens were being preyed upon by hawks that lived in areas around their ranches. The farmers grouped together and hunted the hawks until they were no longer in their area. Which would most likely happen next?,A: The chicken population would go down.\nB: Populations of mice and rats would increase.\nC: Another bird of prey would replace the hawk.\nD: The chickens would have a lower rate of disease.\n
9,A,Copper is an element that is used in electrical wires. What is the smallest unit of copper that still maintains the characteristics of copper?,A: the atom\nB: the electron\nC: the nucleus\nD: the proton\n
